In [5]:
## import pacakge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

labeled_data_file_name = "Labeled_data.csv"
unlabel_data_file_name = "Unlabeled_Data.csv"

label_data_df = pd.read_csv(labeled_data_file_name)
unlabel_data_df = pd.read_csv(unlabel_data_file_name)


In [6]:
## print labeled data
label_data_df.head()

,acctid,state,zip,rollout,cobal,collscr,cs,accessscr,lnacscr,bureauscr,eaglemod,numcalls,numrpcs,totalpay
0,1,CA,93313,Wells,1786,4013181,4,0.1992,-1.61,133,67,104,3,360
1,2,CA,92154,Wells,2048,1015805,1,0.4376,-0.83,109,6,233,6,300
2,3,FL,33319,Wells,5170,4013057,4,0.8140,-0.21,103,90,28,1,10
3,4,WA,98115,Bank_Of_Am,4506,8012265,8,0.2978,-1.21,136,11,8,2,2550
4,5,MI,49615,Associates,2706,3013630,3,0.3722,-0.99,105,42,45,11,2000


In [7]:
## print unlabel data
unlabel_data_df.head()

,acctid,state,zip,rollout,cobal,collscr,cs,accessscr,lnacscr,bureauscr,eaglemod,numcalls,numrpcs
0,3571,CA,91306,Bank_Of_Am,3351,8012287,8,0.4030,-0.91,121,37,183,1
1,3572,FL,33417,Associates,3371,5012983,5,0.1662,-1.79,115,74,106,2
2,3573,CA,92507,Bank_Of_Am,4534,1019323,1,0.6252,-0.47,121,32,75,1
3,3574,CA,92507,Discover,1571,1018457,1,0.4670,-0.76,119,15,16,3
4,3575,AR,72758,Associates,804,5012853,5,0.7620,-0.27,120,12,213,2


# Comapany Background:

Select  Collections,  Inc.,  was  a  start-up  subsidiary  of  a  major credit  card  company

***The company  purchased  distressed  consumer  debt  at  discounted  rates from  such  major  credit  card***
and  then  
used  data-driven  decision-making and dynamic value assessment to optimize the collection processes associated with the purchased accounts

For each purchased account, ***the first decision was whether to resell or attempt to collect***. For accounts the company decided to attempt to collect, a host of tactics was available for use in any sequence and with any frequency. 

Like other collection companies, Select Collections used ***the telephone  and  the  legal  system***  as  its  two  major  collection  tools.  



# Goal:


I want to see if we can ***build a model to predict how much money we will collect from  delinquent  accounts.***  Such  a  model  would  prove  very  useful in  ***deciding which accounts to purchase and how much to pay.***

There are ***3,570*** accounts in a data set that I will call the training set

***We  purchased  these  accounts  and  then  collected  the  amount  shown  in  the  last column, labeled totalpay.*** 
Totalpay is the variable of interest. I want you to ***come up with a way to forecast totalpay.***

I  want  you  to  use  the  model  you  come  up with to predict totalpay for the 3,570 accounts in a separate worksheet of data that I will call the test set. 

I  want  an  Excel  file  from  you  containing a single column of numbers. 
***In cell A1, please put the adjusted R-squared for your model.***
***In cells A2 through A3571, put your predicted values for the accounts in the test data set in ID order.***



# Datadict: 
![alt text](datadict.png "data dict")

# Data description:

![alt text](description.png "data description")

# Colume Comparation to total pay

![alt text](col_comparasion_totalpay.png "colume vs totalpay description")

# Col vs Col description
![alt text](col_to_col_relationship.png "col vs col description")

# Business Understanding


# Understanding the data
#### From data dict :
what col should be care?

#### from data description:
data distrbution is fair ? 
fair in total
rollout is also important ?
not fair

#### from col to total pay part we now only 4 cols are not well distributed

inacscr: The natural log of the ***accessscr***

***accessscr***: The account’s accessibility score, an a priori estimation of how likely it is that the accountholder is reached via phone.
bureauscr: The output of an in-house prediction model based on the likelihood of receiving payments from the accountholder using credit-bureau attributes as predictors 

numcalls: The number of telephone calls made (to date) to the accountholder

numrpcs: The number of “right-party connects” or phone calls in which the collection agent speaks with the accountholder 

#### how about col to col ?

what grid means to me ?


# Data Preparation

From ***Understanding the data parts***, already shown the data related accessibility is important

# Modeling 

#### Representation

What algorithm we are using ? at least it support R-squared

Linear Regression: Linear regression is a simple and commonly used algorithm that predicts a continuous target variable based on one or more independent variables.

Polynomial Regression: Polynomial regression is a form of regression analysis in which the relationship between the independent variable x and the dependent variable y is modelled as an nth degree polynomial. 

#### Optimization

#### Evaluation